In [ ]:
import numpy as np
import h5py
import math
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os, sys
import random
import scipy
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFilter
import time
import io

In [ ]:
# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 process-sim.py <automation_dir> <attpcroot_dir>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET/.sims/0/'
    attpcroot_dir = automation_dir + 'ATTPCROOTv2/'
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # ATTPCROOTv2 directory
    attpcroot_dir = sys.argv[2]

In [ ]:
def indicator_file(file_type, indicator_directory=automation_dir):
    # remove old indicator file(s)
    for file in os.listdir(indicator_directory):
        if file.endswith('.tmp'):
            os.remove(indicator_directory + file)
    
    with open(indicator_directory + file_type + '.tmp', 'w') as f:
        f.write('1')
    if file_type == 'STOP':
        print('STOPPING')
        sys.exit()
    return None

In [ ]:
parameters = pd.read_csv(automation_dir + 'param.csv')

In [ ]:
# ensure that the status is correct

# 0 = queued
# 1 = simulating
# 2 = h5 complete
# 3 = images complete
# 4 = augmentation complete
# 5 = gif complete (final)

indicator_file('AUGMENTATION')

active_sims = parameters[parameters['Status'] == 3]

if len(active_sims) > 1:
    print('More than one simulation marked as active')
    indicator_file('STOP')
elif len(active_sims) == 0:
    print('No simulation images to process')
    sys.exit(0)

image_dir = f'{automation_dir}out/images/'
random.seed(time.time()) 

In [ ]:
# read in augmentation parameters
augVars = { # default values
    'evar': 0,
    'trace_scale': 1,
    'trace_mirror': False,
    'placement_error': 0,
    'track_scale': 1,
    'track_blur': 0,
    'track_noise': 0,
    'track_edge': 2,
    'edge_noise': 0,
    'veto_radius': 66.709,
    'location_shuffle': False,
    'rotate_track': False,
    'mirror_track': False,
    'max_iters' : 100
}

# change based on parameters file
for key in augVars.keys():
    if key in parameters.columns:
        augVars[key] = parameters.loc[active_sims.index[0], key]
if 'Seed' in parameters.columns: # set seed if it is specified
    random.seed(parameters.loc[active_sims.index[0], 'Seed'])

In [ ]:
# pre-processing
image_list = os.listdir(image_dir) # list of images

# generate circle mask for padplane validation
circle_mask = np.ones((145, 145, 4), dtype=np.uint8)
for i in range(circle_mask.shape[0]):
    for j in range(circle_mask.shape[1]):
        if (i-72)**2+(j-72)**2 <= augVars['veto_radius']**2:
            circle_mask[i,j,:] = np.array([0,0,0,0], dtype=np.uint8)

In [ ]:
def fit_check(padplane, circle_mask = circle_mask):
    test_padplane = padplane.copy() * circle_mask[:,:,:3]
    if np.sum(test_padplane[:,:,0]) == np.sum(test_padplane[:,:,2]):
        return True
    else:
        return False

def veto_check(image,circle_mask=circle_mask):
    # apply mask to image
    
    masked_image = image * circle_mask
    padplane = masked_image[0:151,37:188,0] # padplane region

    padplane[np.where(padplane==255)] = 0 # remove white
    
    veto = np.any(padplane != 0) # check for nonzero values
    return veto

In [ ]:
def track_rescale(track, augVars):
    track = Image.fromarray(track)
    width, height = track.size
    scale = augVars['track_scale']
    scale = random.uniform(1/scale, scale)
    new_width = int(width * scale)
    new_height = int(height * scale)
    
    track = track.resize((new_width, new_height))
    blur = augVars['track_blur']
    track = track.filter(ImageFilter.GaussianBlur(random.uniform(0,blur)))

    track = np.array(track)
    return track
def track_noise(track, augVars):
    pad_std = np.std(track[track > 0])
    track[track > 0] += np.random.normal(0, pad_std*augVars['track_noise'], track[track > 0].shape)
    return track

In [ ]:
def aug_padplane(image, circle_mask=circle_mask, augVars=augVars):
    max_iters = 100 # maximum number of iterations to find a valid padplane location
    
    # extract padplane from image
    padplane_bounds = ((3,40),(148,185))
    padplane = image[padplane_bounds[0][0]:padplane_bounds[1][0], padplane_bounds[0][1]:padplane_bounds[1][1], :]
    
    track = padplane.copy()
    track[track == 255] = 0 # remove white background

    track = track[1:, 1:] 
    track = track[::4,::4] # downsample track to remove grid lines

    # extract relative energy of pads
    track = np.average([track[:,:,0].astype(np.float32) / 205, track[:,:,1].astype(np.float32) / 240], axis=0)

    pad_threshold = min(track[track > 0])
    track = np.pad(track, ((10,10),(10,10)), 'constant', constant_values=0) # pad track to prevent edge effects
    
    # PIXEL BASED AUGMENTATIONS
    # rescale track and blur
    if augVars['track_scale'] != 1:
        track = track_rescale((track*255).astype(np.uint8), augVars).astype(np.float32) / 255
    
    # directly add noise to track
    track = track_noise(track, augVars)
    
    # add random firing pixels near edge of track
    # determine what pixels are within edge_range of nonzero pixels
    edge_range = augVars['track_edge']
    edge_pixels = np.zeros(track.shape)
    for i in range(edge_range, track.shape[0] - edge_range):
        for j in range(edge_range, track.shape[1] - edge_range):
            if track[i,j] > 0:
                edge_pixels[i-edge_range:i+edge_range, j-edge_range:j+edge_range] = 1
    edge_pixels = edge_pixels * (1 - (track > 0).astype(bool)) # remove pixels that are already firing
    track += np.random.normal(0, augVars['edge_noise'] * pad_threshold, track.shape) * edge_pixels
    
    
    # reapply threshold to track
    track[track < pad_threshold] = 0
    track[track >= 1] = 1

    # reconstruct track and crop
    track = track.repeat(4, axis=0).repeat(4, axis=1) # upsample track to original size
    track_bounds = np.where(track != 0)
    track_bounds = ((min(track_bounds[0]), max(track_bounds[0])+1), (min(track_bounds[1]), max(track_bounds[1])+1))
    track = track[track_bounds[0][0]:track_bounds[0][1], track_bounds[1][0]:track_bounds[1][1]]
    
     # redraw gridlines
    for i in range(1, track.shape[0]//4):
        track[i*4, :] = 0
    for i in range(1, track.shape[1]//4):
        track[:, i*4] = 0
    track = track[1:, 1:]
    
    # recolor track
    track = np.stack((track*204, track*240, (track > 0) * (track != 1) * 255), axis=2).astype(np.uint8)
    
    # place track in random valid location on padplane
    in_bounds = False
    iters = 0
    while not in_bounds:
        padplane[:,:,0] = padplane[:,:,2]; padplane[:,:,1] = padplane[:,:,2] # blank padplane
        if augVars['location_shuffle']:
            try:
                loc = (random.randint(0, 36 - (track.shape[0] + 1) // 4), random.randint(0, 36 - (track.shape[1] + 1)//4)) # random location

                # insert track into padplane    
                padplane[loc[0]*4+1:loc[0]*4+track.shape[0]+1, loc[1]*4+1:loc[1]*4+track.shape[1]+1,0] += track[:,:,0] # red
                padplane[loc[0]*4+1:loc[0]*4+track.shape[0]+1, loc[1]*4+1:loc[1]*4+track.shape[1]+1,1] += track[:,:,1] # green

                # test for track within radius of padplane
                if fit_check(padplane):
                    in_bounds = True
                else:
                    iters += 1
                    if iters > max_iters:
                        return None
            except:
                iters += 1
                if iters > augVars['max_iters']:
                    return None
        else: # center track on padplane
            loc = (17 - track.shape[0]//8, 17 - track.shape[1]//8) # center shifted by half-track size
            padplane[loc[0]*4+1:loc[0]*4+track.shape[0]+1, loc[1]*4+1:loc[1]*4+track.shape[1]+1,0] += track[:,:,0]
            padplane[loc[0]*4+1:loc[0]*4+track.shape[0]+1, loc[1]*4+1:loc[1]*4+track.shape[1]+1,1] += track[:,:,1]
            
            if fit_check(padplane):
                in_bounds = True
            else:
                return None
    
    # randomly rotate padplane
    if augVars['mirror_track']:
        if random.randint(0,1):
            padplane = np.flip(padplane, 0)
    if augVars['rotate_track']:
        padplane = np.rot90(padplane, random.randint(0,3), (0,1))

    
    # place padplane back into image
    image[padplane_bounds[0][0]:padplane_bounds[1][0], padplane_bounds[0][1]:padplane_bounds[1][1], :] = padplane
    return image

In [ ]:
def aug_trace(image, augVars):
    trace = image[151:,:,0] # extract trace from image
    
    trace = np.sum(255-trace, axis=0).astype(np.int64) # cumulative sum of trace
    
    # find most common non-zero value in trace
    trace_zero = np.bincount(trace[trace > 0]).argmax()
    
    # determine edges of trace
    trace_edges = np.where(trace == trace_zero)
    trace_edges = (trace_edges[0][0], trace_edges[0][-1])
    trace_width = trace_edges[1] - trace_edges[0]
    
    trace[:trace_edges[0]] = trace_zero
    trace[trace_edges[1]:] = trace_zero
    
    trace = trace - trace_zero # set baseline to zero
    
    # peak value in trace_height
    peakx = np.argmax(trace)
    x_trace = np.arange(trace.shape[0]).astype(np.float32)
    
    scale_factor = augVars['trace_scale']**random.uniform(-1,1) # scale trace randomly between 1/trace_scale and trace_scale
    
    
    if augVars['trace_mirror']:
        mirror = random.choice([-1,1]) # randomly mirror trace about peakx
    else:
        mirror = 1

    x_trace = (x_trace - peakx)*scale_factor + peakx # scale trace about peakx
    
    x_trace = x_trace + random.uniform(-augVars['placement_error'], augVars['placement_error']) # randomly shift trace x axis
    
    # crop trace
    crop0 = np.where(x_trace > peakx - trace_width//2 - 1)[0][0]
    crop1 = np.where(x_trace < peakx + trace_width//2 + 1)[0][-1]
    
    x_trace = x_trace[crop0:crop1]
    trace = trace[crop0:crop1]
    
    if mirror == -1:
        x_trace = np.flip(x_trace, 0)
        trace = np.flip(trace, 0)
    
    # export trace as jpg to be loaded as matrix (same process as generating trace originally)
    my_dpi = 96
    fig_size = (224/my_dpi, 73/my_dpi)  # Fig size to be used in the main thread
    fig, ax = plt.subplots(figsize=fig_size)
    ax.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.clear()
    x = np.linspace(0, len(trace)-1, len(trace))
    ax.fill_between(x, trace, color='b', alpha=1)
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=my_dpi)
    fig.clf()
    buf.seek(0)
    with Image.open(buf) as im:
        trace_img_png = np.array(im)
    buf.close()
    
    image[151:,:,:] = trace_img_png[:,:,:3]
    return image

In [ ]:
def aug_ebar(image, augVars):
    # extract energy bar from image
    ebar_bounds = ((5,8),(145,17))
    ebar = image[ebar_bounds[0][0]:ebar_bounds[1][0], ebar_bounds[0][1]:ebar_bounds[1][1], :]

    ebar_slice = np.array([np.mean(ebar[i,1,:]) for i in range(ebar.shape[0])]) # 1d slice of energy bar
    for i in range(ebar_slice.shape[0]):
        if ebar_slice[i] != 255:
            break
    proportion_filled = 1 - (i-1)/ebar_slice.shape[0]
    proportion_filled *= np.random.uniform(1-augVars['evar'], 1+augVars['evar'])

    image[ebar_bounds[0][0]:ebar_bounds[1][0], ebar_bounds[0][1]:ebar_bounds[1][1], :] = 255
    image = fill_energy_bar(image, proportion_filled)
    return image

def blue_range(pad_plane, rows):
	start_row = 140
	low_color = 0
	high_color = 35
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 0] = low_color
		pad_plane[start_row:start_row+5, 8:17, 1] = high_color
		start_row = start_row - 5 
		low_color = low_color + 35
		high_color = high_color + 35
	return pad_plane
def yellow_range(pad_plane, rows):
	start_row = 105
	color = 220
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 2] = color
		start_row = start_row - 5 
		color = color - 15
	return pad_plane
def orange_range(pad_plane, rows):
	start_row = 70
	color = 210
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 1] = color - 15
		pad_plane[start_row:start_row+5, 8:17, 2] = color
		start_row = start_row - 5 
		color = color - 15
	return pad_plane
def red_range(pad_plane, rows):
	start_row = 35
	color = 250
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 0] = color
		pad_plane[start_row:start_row+5, 8:17, 1] = 50
		pad_plane[start_row:start_row+5, 8:17, 2] = 50
		start_row = start_row - 5 
		color = color - 15
	return pad_plane
def fill_energy_bar(image,proportion_filled):
	total_rows = math.floor(proportion_filled * 28) # Calculate how many rows should be filled
	# Fill the energy bar one row at a time
	if total_rows > 0:
		pad_plane = blue_range(image, rows=min(total_rows, 7))
	if total_rows > 7:
		pad_plane = yellow_range(image, rows=min(total_rows-7, 7))
	if total_rows > 14:
		pad_plane = orange_range(image, rows=min(total_rows-14, 7))
	if total_rows > 21:
		pad_plane = red_range(image, rows=min(total_rows-21, 7))
	return image

In [ ]:
for i in range(len(image_list)):
    image = Image.open(image_dir + image_list[i]) # open image
    image = np.array(image)[:,:,:3] # convert to numpy array
    
    image = aug_padplane(image, augVars) # augment padplane
    if image is None:
        continue
    image = aug_trace(image, augVars) # augment trace
    image = aug_ebar(image, augVars) # augment energy bar
    
    # overwrite image
    plt.imsave(image_dir + image_list[i], image)
    plt.close()

In [ ]:
# set status of active sim
parameters.loc[active_sims.index[0], 'Status'] = 4 # complete images
parameters.to_csv(automation_dir + 'param.csv', index=False)

In [ ]:
# attempt to make gif of images
indicator_file('PROCESSING GIF')
sim_name = active_sims['Sim'].values[0]

image_list = os.listdir(automation_dir + 'out/images/')
image_list = [i for i in image_list if i.split('_')[0] == sim_name]
if len(image_list) > 100: # limit number of images to 100
    image_list = image_list[:100]
if len(image_list) < 2:
    sys.exit() # not enough images to make gif
frames = []
for i in range(len(image_list)):
    new_frame = Image.open(automation_dir + "out/images/" + image_list[i])
    frames.append(new_frame)
frames[0].save(f'{automation_dir}out/gifs/{sim_name}.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)

# set status of active sim to 4
parameters.loc[active_sims.index[0], 'Status'] = 5 # complete gif
parameters.to_csv(automation_dir + 'param.csv', index=False)